In [1]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
data = pd.read_csv('alphanet_codetest_daily.csv')
# 定义数据图片开始和结束的时间
time_start = 20190801
time_end = 20211001


stock = pd.DataFrame(data.groupby('ticker')['timestamp'].count() == 487).reset_index()
merge = data.merge(stock, on='ticker', how='left')
merge = merge[merge['timestamp_y'] == True]
merge['timestamp'] = merge['timestamp_x']
merge = merge[['timestamp','ticker','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover']] # feature
merge = merge[(merge['timestamp'] >= time_start) & (merge['timestamp'] <= time_end)]
merge

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover
0,20190923,1,15.34,15.47,15.18,15.38,15.3122,1403282.00,0.2608,0.7428,1.5038
1,20190923,2,26.49,26.49,26.00,26.15,26.1471,603530.46,-2.4254,0.6212,1.0882
2,20190923,4,20.36,20.65,19.61,20.04,20.0171,29893.32,-4.7076,3.6188,5.9394
3,20190923,5,3.37,3.37,3.27,3.30,3.2947,111437.30,-2.0772,1.0533,1.3495
4,20190923,6,5.34,5.40,5.21,5.38,5.2993,136997.47,1.1278,1.0161,1.5887
...,...,...,...,...,...,...,...,...,...,...,...
1958151,20210923,688168,52.80,53.30,52.00,52.45,52.5581,5919.73,1.0792,2.0771,2.1589
1958162,20210923,688188,446.80,449.00,425.20,438.14,438.5992,4444.64,-1.6830,1.6540,1.6540
1958216,20210923,688321,39.20,39.30,38.64,38.90,38.8391,14252.01,0.0772,0.5631,0.6717
1958221,20210923,688333,199.87,201.01,192.01,200.34,196.4810,4126.57,0.6177,0.9092,0.9092


In [2]:
day_back = 3 # 回溯几天进行计算
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天


x , y , x_delay = [] , [], [] # 初始数据集

for count , ticker in enumerate(merge['ticker'].drop_duplicates()):
#     print(count)
    one_data = merge[merge['ticker'] == ticker]
    one_data['pct_change_shift'] = (one_data['close'].shift(-day_back) - one_data['close']) / one_data['close'] * 100
    one_data = one_data.set_index(['timestamp','ticker'])
    one_data = one_data.dropna() # 丢弃因为回溯而产生的空值
    array = np.array(one_data)

    for i in range(0,array.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        x.append(array[i:i+day,:-1].T)
        y.append(array[i+day-1][-1])
x  , y = np.array(x) , np.array(y).reshape(-1,1) # x = (153, 9, 30) , y = (153,1)
x = x.reshape(x.shape[0],1,x.shape[1],x.shape[2]) # x = (153, 1, 9, 30)
print("x.shape: ",x.shape)
print("y.shape: ",y.shape)

C:\Users\Xinfei\AppData\Local\Temp/ipykernel_9316/2247197430.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_data['pct_change_shift'] = (one_data['close'].shift(-day_back) - one_data['close']) / one_data['close'] * 100


x.shape:  (553277, 1, 9, 30)
y.shape:  (553277, 1)


In [3]:
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0],i] for i in l1[1:]]
    l1 = l1[1:]
    return v+generateC(l1)
def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num , num_rev
def data_info(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length = data.shape[3] # 30
    feat_num = data.shape[2] # 9
    """num:组合数对列表,num_rev:num的翻转列表"""
    num , num_rev = generate_Num_and_ReversedNum(feat_num)
    conv_feat = len(num)
    # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
    if data_length % stride == 0:
        step_list = list(range(0,data_length+stride,stride))
    elif data_length % stride<=5:
        mod = data_length % stride
        step_list = list(range(0,data_length-stride,stride))+[data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
    return data_length,feat_num,conv_feat,num,num_rev,step_list
def ts_cov4d(data,stride):
    '''计算4维数据的协方差'''
    '''data:[N,C,H,W],,W:price length,N:batch size'''
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    #计算的过程中务必保持keepdims=True
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,num,start:end] # (2000, 1, 36, 2, 10)
        sub_data2 = data[:,:,num_rev,start:end]
        mean1 = sub_data1.mean(axis = 4,keepdims = True) # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis = 4,keepdims = True)
        spread1 = sub_data1 - mean1 # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1*spread2).sum(axis = 4,keepdims = True)/(sub_data1.shape[4] - 1)).mean(axis = 3,keepdims = True) # (2000, 1, 36, 1, 1)
        l.append(cov)
    corr = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,conv_feat,len(step_list)-1) # (2000, 1, 36, 3)
    return torch.from_numpy(corr)
def ts_corr4d(data,stride,conv1):
    '''计算4维数据的相关系数'''
    '''data:[N,C,H,W],,W:price length,N:batch size'''
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,num,start:end]
        sub_data2 = data[:,:,num_rev,start:end]
        std1 = sub_data1.std(axis = 4,keepdims = True)
        std2 = sub_data2.std(axis = 4,keepdims = True)
        std = (std1*std2).mean(axis = 3,keepdims = True)
        l.append(std)
    std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,conv_feat,len(step_list)-1)
#     cov = ts_cov4d(data,stride)
    fct = (sub_data1.shape[4]-1)/sub_data1.shape[4]
    return (conv1/torch.from_numpy(std))*fct
def ts_stddev4d(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,:,start:end]
        std1 = sub_data1.std(axis = 3,keepdims = True)
        l.append(std1)
    std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
    return torch.from_numpy(std)
def ts_zscore(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,:,start:end]
        mean = sub_data1.mean(axis = 3,keepdims = True)
        std = sub_data1.std(axis = 3,keepdims = True)
#         std[std == 0] = 1e-9
        z_score = mean/std
        l.append(z_score)
    z_score = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
    return torch.from_numpy(z_score)
def ts_return(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    data[data == 0] = 1e-9
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,:,start:end]
        ret = sub_data1[:,:,:,-1]/sub_data1[:,:,:,0] - 1
        l.append(ret)
    z_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
    z_data[z_data > 1] = 1
    return torch.from_numpy(z_data)
def ts_decaylinear(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        time_spread = end - start
        weight = np.arange(1,time_spread+1)
        weight = weight/(weight.sum())
        sub_data1 = (data[:,:,:,start:end]*weight).mean(axis = 3,keepdims = True)
        l.append(sub_data1)
    decay_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
    return torch.from_numpy(decay_data)
def ts_pool(data,stride,method):
    if type(data) == torch.Tensor:
        data = data.detach().numpy()
    if data.shape[-1] <= stride:
        step_list = [0,data.shape[-1]]
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        if method == 'max':
            sub_data1 = data[:,:,:,start:end].max(axis = 3,keepdims = True)
        if method == 'min':
            sub_data1 = data[:,:,:,start:end].min(axis = 3,keepdims = True)
        if method == 'mean':
            sub_data1 = data[:,:,:,start:end].mean(axis = 3,keepdims = True)
        l.append(sub_data1)
    try:
        pool_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list) - 1)
    except:
        pool_data = np.squeeze(np.array(l)).reshape(-1,feat_num,len(step_list) - 1)
    return torch.from_numpy(pool_data)

In [21]:
conv = ts_cov4d(x,10).detach().numpy()
# conv = nn.BatchNorm1d(36,affine=True)(conv)
print(np.max(conv))
print(np.mean(conv))
print(np.median(conv))
print(np.std(conv))

74029173.36078541
12930.02505032905
0.1056851611111112
185265.1201575766


In [22]:
conv = nn.BatchNorm1d(36,affine=True)(torch.from_numpy(conv).to(torch.float))
conv = conv.detach().numpy()
print(np.max(conv))
print(np.mean(conv))
print(np.median(conv))
print(np.std(conv))

298.94443
-2.9307723e-09
-0.06698045
0.9999991


In [4]:
feat_num = 9
# x = x.detach().numpy()
# x
batch = nn.BatchNorm1d(36,affine=True)
batch2 = nn.BatchNorm1d(9,affine=True)
conv1 = ts_cov4d(x,10)
# conv1 = ts_cov4d(x,10).to(torch.float)
# bc1 = batch(conv1)
conv2 = ts_corr4d(x,10,conv1).to(torch.float)
bc2 = batch(conv2)
bc1 = batch(conv1.to(torch.float))
conv3 = ts_stddev4d(x,10).to(torch.float)
bc3 = batch2(conv3)
conv4 = ts_zscore(x,10).to(torch.float)
bc4 = batch2(conv4)
conv5 = ts_return(x,10).to(torch.float)
bc5 = batch2(conv5)
conv6 = ts_decaylinear(x,10).to(torch.float)
bc6 = batch2(conv6)

feat_cat = torch.cat([bc1,bc2,bc3,bc4,bc5,bc6],axis = 1) # 特征聚合
shape = feat_cat.shape
feat_cat = feat_cat.reshape(shape[0],1,shape[1],shape[2])
# Pooling
ts_max = ts_pool(feat_cat ,3,method = 'max')
ts_max.shape
ts_max = nn.BatchNorm1d(108,affine = True)(ts_max)
ts_min = ts_pool(feat_cat ,3,method = 'min')
ts_min = nn.BatchNorm1d(108,affine = True)(ts_min)
ts_mean = ts_pool(feat_cat ,3,method = 'mean')
ts_mean = nn.BatchNorm1d(108,affine = True)(ts_mean)
data_pool = torch.cat([ts_max,ts_min,ts_mean],axis = 1)
data_pool.shape
data_pool = data_pool.flatten(start_dim = 1)
data_pool.shape

In [10]:
conv2[0]

tensor([[ 0.7850,  0.7623,  0.6012],
        [ 0.9554,  0.5325,  0.8029],
        [ 0.7221,  0.3724,  0.3186],
        [ 0.8250,  0.4756,  0.5489],
        [-0.4866,  0.2650, -0.3823],
        [-0.1128, -0.5146, -0.5298],
        [-0.5086,  0.2649, -0.3821],
        [-0.5281,  0.1490, -0.3823],
        [ 0.9237,  0.8145,  0.8038],
        [ 0.9661,  0.7328,  0.8518],
        [ 0.9912,  0.8580,  0.9577],
        [-0.0221,  0.2592,  0.2861],
        [ 0.4470,  0.0725,  0.2058],
        [-0.0604,  0.2592,  0.2861],
        [-0.0957,  0.0927,  0.2861],
        [ 0.8798,  0.8886,  0.7226],
        [ 0.9515,  0.9750,  0.8794],
        [-0.3103, -0.2709, -0.2642],
        [ 0.1433,  0.3720, -0.0887],
        [-0.3422, -0.2709, -0.2642],
        [-0.3711, -0.4249, -0.2643],
        [ 0.9700,  0.9170,  0.9186],
        [-0.0261, -0.2470,  0.2639],
        [ 0.5815,  0.5743,  0.5845],
        [-0.0647, -0.2470,  0.2638],
        [-0.1004, -0.3466,  0.2638],
        [-0.0897, -0.1791,  0.1458],
 

In [28]:
conv2[torch.isnan(conv2) == True].shape

torch.Size([26152])

In [ ]:
class Pooling(object):
    def __init__(self,data,stride):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = data.detach().numpy()
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2] # 9
        self.step_list = self.generate_Step_List(self.data_length,self.stride)
        self.extracted_data = self.Extraction(self.data,self.feat_num,self.stride)
    def Extraction(self,data,feat_num,stride):
        print("------Start Pooling------")
        # Pooling
        ts_max = self.ts_pool(feat_cat,self.stride,self.feat_num,self.step_list,method = 'max')
        ts_max = nn.BatchNorm1d(108,affine = True)(ts_max)
        ts_min = self.ts_pool(feat_cat ,self.stride,self.feat_num,self.step_list,method = 'min')
        ts_min = nn.BatchNorm1d(108,affine = True)(ts_min)
        ts_mean = self.ts_pool(feat_cat ,self.stride,self.feat_num,self.step_list,method = 'mean')
        ts_mean = nn.BatchNorm1d(108,affine = True)(ts_mean)
        data_pool = torch.cat([ts_max,ts_min,ts_mean],axis = 1)
        data_pool = data_pool.flatten(start_dim = 1)
        print("Pooling shape: ",data_pool.shape )
        return data_pool
    def generate_Step_List(self,data_length,stride):
        # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
        if data_length % stride == 0:
            step_list = list(range(0,data_length+stride,stride))
        elif data_length % stride<=5:
            mod = data_length % stride
            step_list = list(range(0,data_length-stride,stride))+[data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
        return step_list
    def ts_pool(self,data,stride,feat_num,step_list,method):
        if type(data) == torch.Tensor:
            data = data.detach().numpy()
        if data.shape[-1] <= stride:
            step_list = [0,data.shape[-1]]
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            if method == 'max':
                sub_data1 = data[:,:,:,start:end].max(axis = 3,keepdims = True)
            if method == 'min':
                sub_data1 = data[:,:,:,start:end].min(axis = 3,keepdims = True)
            if method == 'mean':
                sub_data1 = data[:,:,:,start:end].mean(axis = 3,keepdims = True)
            l.append(sub_data1)
        try:
            pool_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list) - 1)
        except:
            pool_data = np.squeeze(np.array(l)).reshape(-1,feat_num,len(step_list) - 1)
        return torch.from_numpy(pool_data)

In [44]:
convolutional = Convolutional(x,10)
feat_cat = convolutional.extracted_data
# pooling = Pooling(feat_cat,3)
# x = pooling.extracted_data.detach().numpy()
# print("x.shape : ",x.shape)
# print("y.shape : ",y.shape)

------Start Extraction------
------Finished ts_cov4d----output shape:  torch.Size([553277, 36, 3])
------Finished ts_corr4d----output shape:  torch.Size([553277, 36, 3])


Exception: Input data dimensions should be [N,C,H,W]

In [45]:
feat_cat

tensor([[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         ...,
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         ...,
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         ...,
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        ...,

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         ...,
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         ...,
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         ...,
         [nan, nan, nan],
        

In [10]:
from torch.utils.data import Dataset, DataLoader
# x = np.random.uniform(10,100,(3124,1,9,30))
# y = np.random.randn(3124,1)
class Testdataset(Dataset):
    def __init__(self, data,label):
        self.data = torch.from_numpy(data)
        self.label = torch.from_numpy(label)
    def __getitem__(self, index):
        return self.data[index], self.label[index]
    def __len__(self):
        return len(self.data)
trainset = Testdataset(x,y)
trainloader = DataLoader(trainset,batch_size = 500000,shuffle = False)
for i,data in enumerate(trainloader):
    input_data,label = data
    print(i,input_data.size())

0 torch.Size([500000, 324])
1 torch.Size([53277, 324])


In [16]:
import torch
from torch import nn

class AlphaNet(nn.Module):
    def __init__(self,factor_num,fully_connect_layer_neural):
        # super 父类，调用父类的构造，这一步必须有
        # 第一个参数为定义类的名称，第二个为self
        super(AlphaNet,self).__init__()
        self.fc1_neuron = (factor_num * (factor_num -1) + 4 * factor_num)* 3
        self.fc2_neuron = fully_connect_layer_neural
        self.dropout = nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(self.fc1_neuron,self.fc2_neuron)
        self.out = nn.Linear(self.fc2_neuron,1)
        self.relu = nn.ReLU()
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred

In [23]:
import torch.optim as optim
alphanet = AlphaNet(9,30)
print(alphanet)
criterion = nn.MSELoss()
LR = 0.0001
optimizer = optim.RMSprop(alphanet.parameters(), lr=LR, alpha=0.9)
epoch_num = 1
loss_list = []
for epoch in range(epoch_num):
    train_loss = 0.0
    for data,label in trainloader:
        print(data.shape)
        print(label.shape)
        out_put = alphanet(data)
        print("output.shape: ",out_put.shape)
        loss = criterion(out_put,label.to(torch.float))
#         optimizer.zero_grad()
#         train_loss += loss
#         loss.backward()
#         optimizer.step()
#     loss_list.append(train_loss.item())
#     print("current epoch time:",epoch+1,"  Loss: ",train_loss.item())

AlphaNet(
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=324, out_features=30, bias=True)
  (out): Linear(in_features=30, out_features=1, bias=True)
  (relu): ReLU()
)
torch.Size([500000, 324])
torch.Size([500000, 1])
output.shape:  torch.Size([500000, 1])
torch.Size([53277, 324])
torch.Size([53277, 1])
output.shape:  torch.Size([53277, 1])


In [24]:
label

tensor([[ 1.2231],
        [-6.4079],
        [ 2.9343],
        ...,
        [-5.6452],
        [ 5.4701],
        [ 8.9870]], dtype=torch.float64)

In [36]:
feat_cat[0][0]

tensor([[-0.0293, -0.0297, -0.0300],
        [-0.0289, -0.0295, -0.0296],
        [-0.0267, -0.0275, -0.0276],
        [-0.0283, -0.0290, -0.0290],
        [-0.9412,  0.2199, -0.3941],
        [ 0.1667,  0.0964,  0.1174],
        [-0.1594, -0.1389, -0.1490],
        [-0.1582, -0.1341, -0.1445],
        [-0.0286, -0.0293, -0.0294],
        [-0.0290, -0.0301, -0.0301],
        [-0.0296, -0.0305, -0.0306],
        [-0.2876,  0.0312, -0.1063],
        [ 0.0631, -0.0813, -0.0579],
        [-0.2064, -0.2001, -0.2024],
        [-0.2010, -0.1952, -0.1941],
        [-0.0281, -0.0288, -0.0290],
        [-0.0289, -0.0295, -0.0297],
        [-0.6820, -0.4986, -0.2886],
        [ 0.0165,  0.0476, -0.0471],
        [-0.1274, -0.1228, -0.1192],
        [-0.1268, -0.1252, -0.1153],
        [-0.0290, -0.0300, -0.0301],
        [-0.2610, -0.5075, -0.0564],
        [-0.0936, -0.1729, -0.1752],
        [-0.1782, -0.1809, -0.1733],
        [-0.1725, -0.1779, -0.1643],
        [-0.3640, -0.4248, -0.1332],
 

In [68]:
# import torch
# from torch import nn


# class AlphaNet(nn.Module):
#     def __init__(self,factor_num,fully_connect_layer_neural):
#         """ factor_num : 因子个数
#             fully_connect_layer_neural: 全连接层神经元的数量
#         """
#         super(AlphaNet,self).__init__()
#         self.fc1_neuron = (factor_num * (factor_num -1) + 4 * factor_num)* 3
#         self.fc2_neuron = fully_connect_layer_neural
#         self.batchnorm = nn.BatchNorm2d(1,affine=True)
#         self.dropout = nn.Dropout(0.5)
#         self.fc1 = nn.Linear(self.fc1_neuron,self.fc2_neuron)
#         self.out = nn.Linear(self.fc2_neuron,1)
#         self.relu = nn.ReLU()
#     def forward(self,data,num,num_rev):
#         # Conv
#         conv1 = self.ts_cov4d(data,10).to(torch.float)
#         sigmoid1 = torch.sigmoid(conv1)
#         bc1 = self.batchnorm(sigmoid1)
#         conv2 = self.ts_corr4d(data,10).to(torch.float)
#         bc2 = self.batchnorm(conv2)
#         conv3 = self.ts_stddev4d(data,10).to(torch.float)
#         sigmoid3 = torch.sigmoid(conv3)
#         bc3 = self.batchnorm(sigmoid3)
#         conv4 = self.ts_decaylinear(data,10).to(torch.float)
#         sigmoid4 = torch.sigmoid(conv4)
#         bc4 = self.batchnorm(sigmoid4)
#         conv5 = self.ts_zscore(data,10).to(torch.float)
#         bc5 = self.batchnorm(conv5)
#         data_conv = torch.cat([bc1,bc2,bc3,bc4,bc5],axis = 2)
#         # Pooling
#         ts_max = self.ts_pool(data_conv,3,method = 'max')
#         ts_max = self.batchnorm(ts_max)
#         ts_min = self.ts_pool(data_conv,3,method = 'min')
#         ts_min = self.batchnorm(ts_min)
#         ts_mean = self.ts_pool(data_conv,3,method = 'mean')
#         ts_mean = self.batchnorm(ts_mean)
#         data_fin = torch.cat([ts_max,ts_min,ts_mean],axis = 2)
#         data_fin = data_fin.flatten(start_dim = 1)
#         input_size = data_fin.size(1)
#         ful_connect = self.dropout(self.relu(self.fc1(data_fin)))
#         output = self.out(ful_connect)
#         return output.to(torch.float)
#     # 为了cov 和 corr提取并行计算产生计算列表
#     def generateC(l1):
#         if len(l1) == 1:
#             return []
#         v = [[l1[0],i] for i in l1[1:]]
#         l1 = l1[1:]
#         return v+generateC(l1)
#     def generate_Num_and_ReversedNum(feat_nums):
#         list1 = list(range(feat_nums))
#         num = generateC(list1)
#         num_rev = []
#         for l in num:
#             l1 = l.copy()
#             l1.reverse()
#             num_rev.append(l1)
#         return num , num_rev
#     def data_info(data,stride):
#         if len(data.shape)!=4:
#             raise Exception('Input data dimensions should be [N,C,H,W]')
#         data_length = data.shape[3] # 30
#         feat_num = data.shape[2] # 9
#         """num:组合数对列表,num_rev:num的翻转列表"""
#         num , num_rev = generate_Num_and_ReversedNum(feat_num)
#         conv_feat = len(num)
#         # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
#         if data_length % stride == 0:
#             step_list = list(range(0,data_length+stride,stride))
#         elif data_length % stride<=5:
#             mod = data_length % stride
#             step_list = list(range(0,data_length-stride,stride))+[data_length]
#         else:
#             mod = data_length % stride
#             step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
#         return data_length,feat_num,conv_feat,num,num_rev,step_list
#     def ts_cov4d(self,data,stride):
#         '''计算4维数据的协方差'''
#         '''data:[N,C,H,W],,W:price length,N:batch size'''
#         if len(data.shape)!=4:
#             raise Exception('Input data dimensions should be [N,C,H,W]')
#         data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
#         l = []
#         #计算的过程中务必保持keepdims=True
#         for i in range(len(step_list)-1):
#             start = step_list[i]
#             end = step_list[i+1]
#             sub_data1 = data[:,:,num,start:end] # (2000, 1, 36, 2, 10)
#             sub_data2 = data[:,:,num_rev,start:end]
#             mean1 = sub_data1.mean(axis = 4,keepdims = True) # (2000, 1, 36, 2, 1)
#             mean2 = sub_data2.mean(axis = 4,keepdims = True)
#             spread1 = sub_data1 - mean1 # (2000, 1, 36, 2, 10)
#             spread2 = sub_data2 - mean2
#             cov = ((spread1*spread2).sum(axis = 4,keepdims = True)/(sub_data1.shape[4] - 1)).mean(axis = 3,keepdims = True) # (2000, 1, 36, 1, 1)
#             l.append(cov)
#         corr = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,conv_feat,len(step_list)-1) # (2000, 1, 36, 3)
#         return torch.from_numpy(corr)
#     def ts_corr4d(self,data,stride):
#         '''计算4维数据的相关系数'''
#         '''data:[N,C,H,W],,W:price length,N:batch size'''
#         if len(data.shape)!=4:
#             raise Exception('Input data dimensions should be [N,C,H,W]')
#         data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
#         l = []
#         for i in range(len(step_list)-1):
#             start = step_list[i]
#             end = step_list[i+1]
#             sub_data1 = data[:,:,num,start:end]
#             sub_data2 = data[:,:,num_rev,start:end]
#             std1 = sub_data1.std(axis = 4,keepdims = True)
#             std2 = sub_data2.std(axis = 4,keepdims = True)
#             std = (std1*std2).mean(axis = 3,keepdims = True)
#             l.append(std)
#         std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,conv_feat,len(step_list)-1)
#         cov = self.ts_cov4d(data,stride)
#         fct = (sub_data1.shape[4]-1)/sub_data1.shape[4]
#         return (cov/torch.from_numpy(std))*fct
#     def ts_stddev4d(self,data,stride):
#         if len(data.shape)!=4:
#             raise Exception('Input data dimensions should be [N,C,H,W]')
#         data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
#         l = []
#         for i in range(len(step_list)-1):
#             start = step_list[i]
#             end = step_list[i+1]
#             sub_data1 = data[:,:,:,start:end]
#             std1 = sub_data1.std(axis = 3,keepdims = True)
#             l.append(std1)
#         std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,feat_num,len(step_list)-1)
#         return torch.from_numpy(std)
#     def ts_zscore(self,data,stride):
#         if len(data.shape)!=4:
#             raise Exception('Input data dimensions should be [N,C,H,W]')
#         data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
#         l = []
#         for i in range(len(step_list)-1):
#             start = step_list[i]
#             end = step_list[i+1]
#             sub_data1 = data[:,:,:,start:end]
#             mean = sub_data1.mean(axis = 3,keepdims = True)
#             std = sub_data1.std(axis = 3,keepdims = True)
#             z_score = mean/std
#             l.append(z_score)
#         z_score = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,feat_num,len(step_list)-1)
#         return torch.from_numpy(z_score)
#     def ts_decaylinear(self,data,stride):
#         if len(data.shape)!=4:
#             raise Exception('Input data dimensions should be [N,C,H,W]')
#         data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
#         l = []
#         for i in range(len(step_list)-1):
#             start = step_list[i]
#             end = step_list[i+1]
#             time_spread = end - start
#             weight = np.arange(1,time_spread+1)
#             weight = weight/(weight.sum())
#             sub_data1 = (data[:,:,:,start:end]*weight).mean(axis = 3,keepdims = True)
#             l.append(sub_data1)
#         decay_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,feat_num,len(step_list)-1)
#         return torch.from_numpy(decay_data)
#     def ts_pool(self,data,stride,method):
#         if type(data) == torch.Tensor:
#             data = data.detach().numpy()
#         if data.shape[-1] <= stride:
#             step_list = [0,data.shape[-1]]
#         if len(data.shape)!=4:
#             raise Exception('Input data dimensions should be [N,C,H,W]')
#         data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
#         l = []
#         for i in range(len(step_list)-1):
#             start = step_list[i]
#             end = step_list[i+1]
#             if method == 'max':
#                 sub_data1 = data[:,:,:,start:end].max(axis = 3,keepdims = True)
#             if method == 'min':
#                 sub_data1 = data[:,:,:,start:end].min(axis = 3,keepdims = True)
#             if method == 'mean':
#                 sub_data1 = data[:,:,:,start:end].mean(axis = 3,keepdims = True)
#             l.append(sub_data1)
#         try:
#             pool_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,feat_num,len(step_list) - 1)
#         except:
#             pool_data = np.squeeze(np.array(l)).reshape(-1,1,feat_num,len(step_list) - 1)
#         return torch.from_numpy(pool_data)